In [ ]:

#     В ЭТОМ КОДЕ:

#     + кластеризуем посты в группе ССММ (про выборы/нет, анонсы/результаты выборов)
#     + посты с анонсами выборов конвертируем в словарики (имя, курс-поток-группа, текст программы)
#     + посты с результатами выборов конвертируем в словарики со всей информацией об итогах выборов
#     + последние два пункта адаптировать для постов про магистров
#   !!! расставляем флаги предпочтений кандидатов, исходя из текста программ


In [2]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [ ]:
'''
ЧТО ПОФИКСИТЬ:
- БАГ: путает блок текста в шапке репоста с блоком текста в самом репосте (пример: https://vk.com/wall-33327655_2261)
- находить посты с перечнем избранных должностей
'''

# СДЕЛАТЬ ПАРСИНГ ОБСУЖДЕНИЙ В ГРУППЕ:
#     треды с кандидатами 1-3 созывов
#     треды с отчетами о собраний 1-2(3?) созывов


In [275]:
# ссылка на репост без шапки, репостнутая запись - без текста (обновление аватарки):

req_announcement = requests.get('https://vk.com/wall-33327655_2261')
html_doc = req_announcement.text
print(html_doc)

# типы постов:

#     https://vk.com/wall-33327655_2250 -- пост с тегом
#     https://vk.com/wall-33327655_2261 -- пост без тега
#     https://vk.com/wall-33327655_2255 -- post not found
#     https://vk.com/wall-33327655_2262 -- коммент под постом с тегом
#     https://vk.com/wall-33327655_3330 -- репост без шапки поста с адресом 104154624_1
#     https://vk.com/wall-33327655_8489 -- репост с шапкой поста по адресу 77014399_6331


            <!DOCTYPE html>
      <html lang="ka" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="Всем! Всем! Всем!" />
        <meta property="og:description" content="Всем! Всем! Всем!" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-touch-icon" href="/images/icons/pwa/apple/defa

## CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK

In [3]:
'''
CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK
'''

def vk_post_info_by_link(link_vk, language):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
#     дефолтные значения
    content_type = 'no text'
    content_subtype = 'other'
    parent_link = ''
    text_lines = []
    link_year_no = 0
    link_month_no = 0
    link_day_no = 0
    link_date = date(1970, 1, 1)
    link_date_time = ''
    link_time = ''
    months_dict = {
        'russian': ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'],
        'english': ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
#         https://en.wikipedia.org/wiki/Georgian_calendar
        'georgian': ['იან', 'თებ', 'მარ', 'აპრ', 'მაი', 'ივნ', 'ივლ', 'აგვ', 'სექ', 'ოქტ', 'ნოე', 'დეკ']
    }
    months = months_dict[language]

    # ПРОВЕРЯЮ, СУЩЕСТВУЕТ ЛИ ПОСТ
    #     <div class="service_msg service_msg_error">Запись не найдена.</div>
    flg_service_msg_error = soup.find('div', attrs={'class': 'service_msg service_msg_error'})
    if flg_service_msg_error:
        link_type = 'page not found'
    else:
        link_type = 'post'
        # когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
        # когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />
        
        # ПРОВЕРЯЮ, НЕ КОММЕНТ ЛИ ЭТО
        rel_canonical = soup.find('link', attrs={'rel': 'canonical'})
        flg_rel_canonical_reply = re.findall('reply=', str(rel_canonical))
#             + ДОБАВИТЬ ПОЛЕ МАТЕРИНСКОЙ ССЫЛКИ
        if flg_rel_canonical_reply:
            link_type = 'comment'
            
        # ПРОВЕРЯЮ, НЕ РЕПОСТ ЛИ ЭТО
        flg_repost = soup.find('div', attrs={'data-copy': True})
        if flg_repost:
            link_type = 'repost'
#             <div class="wall_item single_wall_item bl_item mark_top_verified" data-copy='-77014399_6331'>
            parent_link = 'https://vk.com/wall' + flg_repost['data-copy']
    
    if link_type in ['post','repost']:
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
        # <div class="wi_body">
        #     <div class="pi_text">

        wi_body = soup.find('div', attrs={'class': 'wi_body'})
        if wi_body is None:
            announcement_list = []
            result_list = []
        else:
            pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
            if pi_text is None:
                announcement_list = []
                result_list = []
            else:
                text_lines = [line for line in pi_text.stripped_strings]
                    # определить тип контента
                    # анонсы: 'Орфография и пунктуация сохранены.'
                    # результаты: 'Таким образом, выборы признаются состоявшимися'
                announcement_list = ['announcement' for line in pi_text.stripped_strings if line.find('Орфография и пунктуация сохранены.') >= 0]
                result_list = ['result' for line in pi_text.stripped_strings if line.find('Таким образом, выборы признаются состоявшимися.') >= 0]
                content_type = 'text'
                if any(election_tag in text_lines for election_tag in ['#выборы@stud_mm','#выборы_ссмм']):
                    content_type = 'election'
    
                if len(announcement_list) > 0:
                    content_subtype = 'announcement'
                elif len(result_list) > 0:
                    content_subtype = 'result'
        
        # ВЫТАСКИВАЮ ДАТУ
        # <div class="wi_info">
        #     <span class="wi_date">
        wi_info = soup.find('div', attrs={'class': 'wi_info'})
        if wi_info:
            wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
            date_list = [date for date in wi_date.stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
            link_date_time = date_list[0]
            link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
        
    if link_type == 'comment':
        reply_link = link_vk[link_vk.index('_')+1:]
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ КОММЕНТА ПОСТРОЧНО    
#             <div class="ReplyItem__body">    180% голосов. Это выборы на мехмате? Это проценты чего?</div>
#             <div class="ReplyItem__date">
#             <a class="item_date" href="https://m.vk.com/wall-33327655_2254?reply=2262#reply2262" rel="noopener">22 &#4316;&#4317;&#4308;&#4315; 2014</a></div>
        reply_body = soup.find('div', attrs={'class': 'ReplyItem__body'})
        reply_bodies = soup.find_all('div', attrs={'class': 'ReplyItem__body'})
        reply_dates = soup.find_all('a', attrs={'class': 'item_date'})
        reply_links = [ff['href'][ff['href'].index('#reply')+6:] for ff in soup.find_all('a', attrs={'class': 'item_date'})]
        reply_no = reply_links.index(reply_link)
        if reply_body is None:
            announcement_list = []
            result_list = []
        else:
            content_type = 'text'
            text_lines = [line for line in reply_bodies[reply_no].stripped_strings]
            date_list = [date for date in reply_dates[reply_no].stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
        parent_link = rel_canonical['href'][: rel_canonical['href'].index('?')]
    
    dict_link = {}
    dict_link[link_vk] = {
        'link_type': link_type, 
        'content_type': content_type,
        'content_subtype': content_subtype,
        'parent_link': parent_link,
        
        'link_date': link_date, 
        'link_year_no': link_year_no, 
        'link_month_no': link_month_no, 
        'link_day_no': link_day_no, 
        'link_date_time': link_date_time, 
        'link_time': link_time, 
        
        'list_of_lines': text_lines
    }
    
    return dict_link


In [274]:
# тест
# vk_post_info_by_link('https://vk.com/wall-33327655_5106', 'georgian') # -- пост с анонсом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2262', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2263', 'georgian') # -- коммент под постом +
vk_post_info_by_link('https://vk.com/wall-33327655_3330', 'georgian') # -- репост без шапки +
# vk_post_info_by_link('https://vk.com/wall-33327655_2261', 'georgian') # -- репост без шапки - ПОЧЕМУ ТУТ text, а не no text
# vk_post_info_by_link('https://vk.com/wall-33327655_8489', 'georgian') # -- репост с шапкой +


{'https://vk.com/wall-33327655_3330': {'link_type': 'repost',
  'content_type': 'no text',
  'content_subtype': 'other',
  'parent_link': 'https://vk.com/wall-104154624_1',
  'link_date': datetime.date(2015, 10, 18),
  'link_year_no': 2015,
  'link_month_no': 10,
  'link_day_no': 18,
  'link_date_time': '18 ოქტ 2015  11:00',
  'link_time': '11:00',
  'list_of_lines': []}}

## functions for processing of posts about elections

### functions for posts with announcements

In [4]:

# вытаскивает из строки номера курса и потока (в предположении, что они в этой строке есть)

# для обычных потоков
def year_stream_no_by_line(line):
    year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line))[0]
    year_stream_line_d = re.sub('\D', '', year_stream_line)
    year_no = int(year_stream_line_d[0])
    stream_no = int(year_stream_line_d[1])
    year_stream_no = str(year_no) + '.' + str(stream_no)
    return year_stream_no, year_no, stream_no

# для магистратуры
def magister_year_by_line(line):
#     если номер года записан цифрами
    if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line))[0]
#         print(year_stream_line)
        year_no = int(year_stream_line[1])
        year_stream_no = 'm' + str(year_no)
#     если номер года записан буквами
    elif re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line))[0]
#         print(year_stream_line)
        year_no = 0
        if str.lower(year_stream_line[1]) == 'перв':
            year_no = 1
        elif str.lower(year_stream_line[1]) == 'втор':
            year_no = 2
        year_stream_no = 'm' + str(year_no)
    return year_stream_no, year_no

# примеры для магистратуры:
# МАГИСТРЫ ПЕРВОГО ГОДА ОБУЧЕНИЯ (выборы завтра на второй паре в аудитории 12-03) +
# МАГИСТРЫ, 2 ГОД ОБУЧЕНИЯ +
# МАГИСТРЫ 2 ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ 1-ГО ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ, 1 КУРС +

# line = 'МАГИСТРЫ, 1 КУРС'
# magister_year_by_line(line)

In [5]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО КАНДИДАТОВ (по списку строк с текстом)


# по списку строк с текстом составляем СЛОВАРЬ: имя: курс-поток, группа, номер первой строки в тексте
def stream_nomination_by_list(list_of_lines):
    nominated_dict = {}
#     year_stream_no, year_no, stream_no = '', 0, 0
    
    for i in range(len(list_of_lines)):
        
#         if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
#             year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
            
#         В НАЧАЛЕ ШАБЛОНА БЫЛО \s (убрал, так как иногда забывают пробел, а алгоритм всё равно жадный)
        if len(re.findall(r'[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i])) > 0:
            name_line_no = i
            name_line = (re.findall(r'[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i]))[0]
            name_list = re.split(', ',name_line)
            name = (re.sub('\W\s', '', name_list[0])).strip().title()
            academic_group_no = int(re.sub('\D', '', name_list[1]))
            another_one_dict = {}
            another_one_dict[name] = {
#                 'year_stream_no': year_stream_no, 
#                 'year_no': year_no, 
#                 'stream_no': stream_no, 
                'academic_group_no': academic_group_no, 
                'name_line_no': name_line_no
            }
            nominated_dict = {**nominated_dict, **another_one_dict}
            
    # первый и последний индексы программы кандидата в списке
    name_list = list(nominated_dict.keys())
    start_index_list = [nominated_dict[key]['name_line_no'] for key in list(nominated_dict.keys())]
    last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
    last_index_list.append(len(list_of_lines)-1)
    
    dict_of_nominated_upd = {}
    
    for name in name_list:
        dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
    # обновить словарь: имя: номер группы, текст программы
    for name in dict_of_nominated_upd:
        program_lines = []

        for k in range(dict_of_nominated_upd[name][0]+1, dict_of_nominated_upd[name][1]+1):
            program_lines.append(list_of_lines[k])
        program_txt = '\n'.join(program_lines)

        nominated_dict[name].update({'program_txt': program_txt})
        del nominated_dict[name]['name_line_no']

    return nominated_dict


# вся инфа про номинации на потоке
def stream_nomination_info_by_dict(list_of_lines):
    year_stream_no = ''
    year_no, stream_no = 0, 0
    
    for i in range(len(list_of_lines)):
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
                
    dict_nominations = stream_nomination_by_list(list_of_lines)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'dict_nominations': dict_nominations
    }
    
    return dict_stream_elections



### functions for posts with results

In [8]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО РЕЗУЛЬТАТЫ ВЫБОРОВ (по списку строк с текстом)


# по списку строк с кандидатами составляем СЛОВАРЬ: имя: голоса, статус
def stream_election_result_by_list(nominated_with_votes):
    dict_nominated = {}

    for line in nominated_with_votes:
        name = ((re.findall(r'[\w\s]+', line))[0]).strip().title()
        votes_line = (re.findall(r'\s[\d]+\sголос', line))[0]
        votes_cnt = int(re.sub('\D', '', votes_line))
        dict_nominated[name] = {'votes_cnt': votes_cnt}
        if (re.findall(r' избран', line)) and not (re.findall(r' не избран', line)):
            elected_flg = 1
        else:
            elected_flg = 0
        dict_nominated[name].update({'elected_flg': elected_flg})
    return dict_nominated


# вся инфа про результаты выборов на потоке
def stream_election_info_by_dict(list_of_lines):
    nominated_with_votes = []
    year_stream_no = ''
    year_no, stream_no = 0, 0
    voted_cnt, stream_size, invalid_cnt, NOTA_cnt = 0, 0, 0, 0
    
    for i in range(len(list_of_lines)):
                    
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
            
        if re.findall('Явка: ', list_of_lines[i]):
            turnout_line = (re.findall(r'[\d]+\s[\w]+\sиз\s[\d]+', list_of_lines[i]))[0]
            voted_cnt = int(((re.findall(r'[\d]+\s', turnout_line))[0]).strip())
            stream_size = int(((re.findall(r'\s[\d]+', turnout_line))[0]).strip())
            
        if len(re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i])) == 0: # отдельно, т.к. может такой записи не быть
            pass
        else:
            invalid_line_no = i
            invalid_line = (re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i]))[0]
            invalid_cnt = re.sub('\D', '', invalid_line)
            
        if re.findall('Против всех', list_of_lines[i]):
            NOTA_line_no = i
            NOTA_line = (re.findall(r'\s[\d]+\sголос', list_of_lines[i]))[0]
            NOTA_cnt = re.sub('\D', '', NOTA_line)
            
        if re.findall('[\w]+\:\s[\d]+\sголос', list_of_lines[i]) and not (re.findall('Против всех', list_of_lines[i])):
            nominated_with_votes.append(list_of_lines[i])
            
        if re.findall('Подтверждено ответственн', list_of_lines[i]):
            organizer_line = (re.findall(r'за\sвыборы\s[\w\s,]+', list_of_lines[i]))[0]
            organizer_line = organizer_line[10::]
            organizer_list = re.split(', | и ',organizer_line)
    
    dict_nominated_with_votes = stream_election_result_by_list(nominated_with_votes)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'stream_size': stream_size, 
        'voted_cnt': voted_cnt,
        'invalid_cnt': invalid_cnt,
        'NOTA_cnt': NOTA_cnt,
        'organizers': organizer_list,
        'dict_nominated_with_votes': dict_nominated_with_votes
    }
    
    return dict_stream_elections


## Processing of all election posts for 4-11 convocations

### launching

In [12]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С АНОНСАМИ ВЫБОРОВ
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ

from itertools import chain

announcement_links = {}
result_links = {}

for num in chain(
    range(2220, 2255) #, # test
#     range(2200, 2360) #, # созыв 4
#     range(3140, 3500) #, # созыв 5
#     range(5040, 5360) #, # созыв 6
#     range(6540, 6720) #, # созыв 7
#     range(7470, 7780), # созыв 8
#     range(8240, 8400), # созыв 9
#     range(9100, 9400), # созыв 10
#     range(9800, 10040) # созыв 11
):
    link = 'https://vk.com/wall-33327655_' + str(num)

    test_dict = vk_post_info_by_link(link, 'georgian')
    print('{0} : {1} \t\t/ {2} \t\t/ {3}'.format(link, test_dict[link]['link_type'], test_dict[link]['content_type'], test_dict[link]['content_subtype']))

    for d in test_dict.values():
        if d['content_type'] == 'election' and d['content_subtype'] == 'announcement':
            announcement_links = {**announcement_links, **test_dict}
        if d['content_type'] == 'election' and d['content_subtype'] == 'result':
            result_links = {**result_links, **test_dict}


for link in announcement_links:
    announcement_links[link].update({'nominated_info': stream_nomination_info_by_dict(announcement_links[link]['list_of_lines'])})
    del announcement_links[link]['list_of_lines']
    
for link in result_links:
    result_links[link].update({'stream_election_info': stream_election_info_by_dict(result_links[link]['list_of_lines'])})
    del result_links[link]['list_of_lines']

print(announcement_links)
print(result_links)



https://vk.com/wall-33327655_2220 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2221 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2222 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2223 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2224 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2225 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2226 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2227 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2228 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2229 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2230 : post 		/ election 		/ other
https://vk.com/wall-33327655_2231 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2232 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2233 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2234 : post 		/ elec

### creating tables about candidates and streams

In [137]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

announcement_streams = {}
for link in announcement_links.keys():
    announcement_streams[link] = {
        'year_x_year_stream': str(announcement_links[link]['link_year_no']) + '_' + announcement_links[link]['nominated_info']['year_stream_no'],
        'link_date': announcement_links[link]['link_date'],
        'link_year_no': announcement_links[link]['link_year_no'], 
        'link_month_no': announcement_links[link]['link_month_no'], 
        'link_day_no': announcement_links[link]['link_day_no'], 
        'link_time': announcement_links[link]['link_time'], 
        'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
        'year_no': announcement_links[link]['nominated_info']['year_no'], 
        'stream_no': announcement_links[link]['nominated_info']['stream_no']
    }
# announcement_streams

announcement_streams_df =  pd.DataFrame(announcement_streams.items(), columns=['link', 'params'])
announcement_streams_df_2 = pd.concat([announcement_streams_df.drop(['params'], axis=1), announcement_streams_df['params'].apply(pd.Series)], axis=1)
announcement_streams_df_3 = announcement_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
announcement_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_9970,2021_1.1,2021-10-15,2021,10,15,11:11,1.1,1,1
1,https://vk.com/wall-33327655_9950,2021_1.2,2021-10-14,2021,10,14,10:20,1.2,1,2
2,https://vk.com/wall-33327655_9969,2021_1.3,2021-10-15,2021,10,15,10:44,1.3,1,3
3,https://vk.com/wall-33327655_9936,2021_2.1,2021-10-13,2021,10,13,11:44,2.1,2,1
4,https://vk.com/wall-33327655_9901,2021_2.2,2021-10-12,2021,10,12,10:40,2.2,2,2
5,https://vk.com/wall-33327655_9932,2021_2.3,2021-10-13,2021,10,13,11:21,2.3,2,3
6,https://vk.com/wall-33327655_9900,2021_2.5,2021-10-12,2021,10,12,10:10,2.5,2,5
7,https://vk.com/wall-33327655_9894,2021_3.1,2021-10-11,2021,10,11,11:48,3.1,3,1
8,https://vk.com/wall-33327655_9889,2021_3.2,2021-10-08,2021,10,8,11:44,3.2,3,2
9,https://vk.com/wall-33327655_9895,2021_3.3,2021-10-11,2021,10,11,11:58,3.3,3,3


In [ ]:
# !!!! РАЗОБРАТЬСЯ С ПОВТОРНЫМИ ПОСТАМИ ВНУТРИ ОДНОГО ПОТОКА


In [138]:
announcement_streams_df_3.to_csv('ssmm_announcement_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [139]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

announcement_names = {}
for link in announcement_links.keys():
    for name in announcement_links[link]['nominated_info']['dict_nominations'].keys():
        announcement_names[name] = {
            'link_year_no_x_name': str(announcement_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': announcement_links[link]['link_year_no'], 
            'year_no': announcement_links[link]['nominated_info']['year_no'], 
            'stream_no': announcement_links[link]['nominated_info']['stream_no'], 
            'academic_group_no': announcement_links[link]['nominated_info']['dict_nominations'][name]['academic_group_no'], 
            'program_txt': announcement_links[link]['nominated_info']['dict_nominations'][name]['program_txt'],
            'link': link
        }
# announcement_names

announcement_names_df =  pd.DataFrame(announcement_names.items(), columns=['name', 'params'])
announcement_names_df_2 = pd.concat([announcement_names_df.drop(['params'], axis=1), announcement_names_df['params'].apply(pd.Series)], axis=1)
announcement_names_df_2.head(10)


,name,link_year_no_x_name,link_year_no,year_no,stream_no,academic_group_no,program_txt,link
0,Малиновский Георгий Андреевич,2021_Малиновский_Георгий_Андреевич,2021,5,4,531,Всем привет!​\nЯ избираюсь в Студенческий сове...,https://vk.com/wall-33327655_9853
1,Мельникова Дарья Владимировна,2021_Мельникова_Дарья_Владимировна,2021,5,4,531,Всем привет!\nВ Студенческий совет избираюсь в...,https://vk.com/wall-33327655_9853
2,Третьяков Петр Максимович,2021_Третьяков_Петр_Максимович,2021,5,3,523,"I, Petr Tretyakov, stand here today, humbled b...",https://vk.com/wall-33327655_9858
3,Клюев Андрей Игоревич,2021_Клюев_Андрей_Игоревич,2021,5,3,523,Избираюсь в Студсовет четвёртый раз. До этого ...,https://vk.com/wall-33327655_9858
4,Евдокимова Софья Александровна,2021_Евдокимова_Софья_Александровна,2021,5,3,523,"В студсовет иду в третий раз, хотелось бы прод...",https://vk.com/wall-33327655_9858
5,Пчельникова Елена Павловна,2021_Пчельникова_Елена_Павловна,2021,5,2,507,"В любом деле, как мне кажется, даже минимально...",https://vk.com/wall-33327655_9860
6,Шереметьев Владислав Валерьевич,2021_Шереметьев_Владислав_Валерьевич,2021,4,3,425,Всем салют!\nКто я?\nЗаконопослушный парень с ...,https://vk.com/wall-33327655_9882
7,Манахова Анастасия Николаевна,2021_Манахова_Анастасия_Николаевна,2021,4,3,423,Всем привет! Я - Настя)\nВ прошлом созыве из-з...,https://vk.com/wall-33327655_9882
8,Водолеев Марк Дмитриевич,2021_Водолеев_Марк_Дмитриевич,2021,3,5,341,"Меня зовут Марк Водолеев, и я уже больше года ...",https://vk.com/wall-33327655_9887
9,Лоншаков Никита Артемович,2021_Лоншаков_Никита_Артемович,2021,3,5,341,Привет! В предыдущем учебного году мне выдалас...,https://vk.com/wall-33327655_9887


In [ ]:
# присвоить ssmm_id в порядке публикации постов (дата, время)
# ИЛИ другим способом ОТСОРТИРОВАТЬ КАНДИДАТОВ (сейчас они в порядке появления в постах в порядке выхода постов, вроде норм)

In [140]:
announcement_names_df_2.to_csv('ssmm_candidates_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [151]:
# full table for 4-11 convocations:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_announcement_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_announcement_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_announcement_info


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_2220,2014_1.1,2014-11-18,2014,11,18,23:10,1.1,1,1
1,https://vk.com/wall-33327655_2225,2014_1.2,2014-11-19,2014,11,19,21:07,1.2,1,2
2,https://vk.com/wall-33327655_2234,2014_1.3,2014-11-21,2014,11,21,1:42,1.3,1,3
3,https://vk.com/wall-33327655_2227,2014_2.1,2014-11-19,2014,11,19,21:12,2.1,2,1
4,https://vk.com/wall-33327655_2314,2014_2.1,2014-11-26,2014,11,26,0:00,2.1,2,1
...,...,...,...,...,...,...,...,...,...,...
153,https://vk.com/wall-33327655_9858,2021_5.3,2021-10-05,2021,10,5,10:35,5.3,5,3
154,https://vk.com/wall-33327655_9853,2021_5.4,2021-10-04,2021,10,4,10:47,5.4,5,4
155,https://vk.com/wall-33327655_9978,2021_6.1,2021-10-16,2021,10,16,10:25,6.1,6,1
156,https://vk.com/wall-33327655_9982,2021_6.3,2021-10-16,2021,10,16,10:50,6.3,6,3


In [152]:
ssmm_announcement_info.to_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


In [153]:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_candidates_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_candidates_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_candidates_info


,name,link_year_no_x_name,link_year_no,year_no,stream_no,academic_group_no,program_txt,link
0,Суркова Анастасия Юрьевна,2014_Суркова_Анастасия_Юрьевна,2014,2,2,209,"«Подавая заявку в прошлом году в студсовет, яв...",https://vk.com/wall-33327655_2202
1,Денисов Илья Олегович,2014_Денисов_Илья_Олегович,2014,2,2,207,"«Вечер добрый, коллеги!\nЯ бы хотел подать зая...",https://vk.com/wall-33327655_2202
2,Тюрина Татьяна Игоревна,2014_Тюрина_Татьяна_Игоревна,2014,2,2,209,«Уже во второй раз избираюсь в студенческий со...,https://vk.com/wall-33327655_2202
3,Дорман Елена Вячеславовна,2014_Дорман_Елена_Вячеславовна,2014,2,3,226,"«По моему мнению, за прошедший год я не плохо ...",https://vk.com/wall-33327655_2203
4,Оськин Александр Андреевич,2014_Оськин_Александр_Андреевич,2014,2,3,225,«Прежде всего здравствуйте:)\nНа протяжении по...,https://vk.com/wall-33327655_2203
...,...,...,...,...,...,...,...,...
357,Майданик Анна Кирилловна,2021_Майданик_Анна_Кирилловна,2021,1,1,107,"Добрый день!\nЧто может быть интереснее, чем у...",https://vk.com/wall-33327655_9970
358,Кадников Климентий Сергеевич,2021_Кадников_Климентий_Сергеевич,2021,1,1,101,"Тэкс.. Значит студсовет, да.\nНу, увидев меня ...",https://vk.com/wall-33327655_9970
359,Мазитова Вероника Рустэмовна,2021_Мазитова_Вероника_Рустэмовна,2021,6,1,606,"Всем привет! Попав в студенческий совет, покин...",https://vk.com/wall-33327655_9978
360,Терехов Михаил Сергеевич,2021_Терехов_Михаил_Сергеевич,2021,6,1,602,За правду!,https://vk.com/wall-33327655_9978


In [154]:
ssmm_candidates_info.to_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


 ### creating tables about results

In [ ]:
# НАПИСАТЬ НОВЫЙ КОД ПО АНАЛОГИИ С АНОНСАМИ

In [15]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

result_streams = {}
for link in result_links.keys():
    result_streams[link] = {
        'year_x_year_stream': str(result_links[link]['link_year_no']) + '_' + result_links[link]['stream_election_info']['year_stream_no'],
        'link_date': result_links[link]['link_date'],
        'link_year_no': result_links[link]['link_year_no'], 
        'link_month_no': result_links[link]['link_month_no'], 
        'link_day_no': result_links[link]['link_day_no'], 
        'link_time': result_links[link]['link_time'], 
        'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
        'year_no': result_links[link]['stream_election_info']['year_no'], 
        'stream_no': result_links[link]['stream_election_info']['stream_no'], 
        'stream_size': result_links[link]['stream_election_info']['stream_size'], 
        'voted_cnt': result_links[link]['stream_election_info']['voted_cnt'], 
        'invalid_cnt': result_links[link]['stream_election_info']['invalid_cnt'], 
        'NOTA_cnt': result_links[link]['stream_election_info']['NOTA_cnt'], 
        'organizers': result_links[link]['stream_election_info']['organizers']
    }
# announcement_streams

result_streams_df =  pd.DataFrame(result_streams.items(), columns=['link', 'params'])
result_streams_df_2 = pd.concat([result_streams_df.drop(['params'], axis=1), result_streams_df['params'].apply(pd.Series)], axis=1)
result_streams_df_3 = result_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
result_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_2250,2014_1.1,2014-11-21,2014,11,21,22:15,1.1,1,1,166,129,10,5,"[Ильёй Денисовым, Анастасией Сурковой]"
1,https://vk.com/wall-33327655_2252,2014_1.2,2014-11-21,2014,11,21,22:25,1.2,1,2,136,95,4,6,"[Ираклием Глунчадзе, Анастасией Сурковой]"
2,https://vk.com/wall-33327655_2254,2014_1.3,2014-11-21,2014,11,21,22:35,1.3,1,3,138,83,1,2,"[Ираклием Глунчадзе, Анастасией Савченко]"
3,https://vk.com/wall-33327655_2246,2014_3.4,2014-11-21,2014,11,21,22:10,3.4,3,4,48,24,0,0,[Марией Митькиной]
4,https://vk.com/wall-33327655_2240,2014_4.1,2014-11-21,2014,11,21,21:45,4.1,4,1,102,52,0,1,"[Анастасией Сурковой, Татьяной Тюриной]"
5,https://vk.com/wall-33327655_2241,2014_4.2,2014-11-21,2014,11,21,21:50,4.2,4,2,102,54,0,0,"[Еленой Дорман, Татьяной Тюриной]"
6,https://vk.com/wall-33327655_2242,2014_4.3,2014-11-21,2014,11,21,22:00,4.3,4,3,82,43,0,9,[Алексеем Матвеевым]
7,https://vk.com/wall-33327655_2253,2014_4.4,2014-11-21,2014,11,21,22:30,4.4,4,4,25,16,0,0,[Ильёй Денисовым]
8,https://vk.com/wall-33327655_2251,2014_5.1,2014-11-21,2014,11,21,22:20,5.1,5,1,96,50,0,0,[Марией Митькиной]
9,https://vk.com/wall-33327655_2243,2014_5.2,2014-11-21,2014,11,21,22:05,5.2,5,2,70,35,0,0,[Алексеем Матвеевым]


In [ ]:
# 'https://vk.com/wall-33327655_2240': 
# {
#     'link_type': 'post', 
#  'content_type': 'election', 
#     'content_subtype': 'result', 'parent_link': '', 'link_date': datetime.date(2014, 11, 21), 
#     'link_year_no': 2014, 'link_month_no': 11, 'link_day_no': 21, 'link_date_time': '21 ნოემ 2014  21:45', 
#     'link_time': '21:45', 
#     'stream_election_info': {
#         'year_stream_no': '4.1', 'year_no': 4, 'stream_no': 1, 
#         'stream_size': 102, 'voted_cnt': 52, 'invalid_cnt': '0', 'NOTA_cnt': '1', 
#         'organizers': ['Анастасией Сурковой', 'Татьяной Тюриной'], 
#         'dict_nominated_with_votes': {'Хапкин Артём Владимирович': {'votes_cnt': 32, 'elected_flg': 1}
        

In [ ]:
# СДЕЛАТЬ ОТДЕЛЬНУЮ ТАБЛИЦУ ДЛЯ ОРГАНИЗАТОРОВ И ПОВЕРНУТЬ ТАК, ЧТОБЫ КАЖДЫЙ ОРГ БЫЛ НА СВОЕЙ СТРОКЕ

In [16]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

result_names = {}
for link in result_links.keys():
    for name in result_links[link]['stream_election_info']['dict_nominated_with_votes'].keys():
        result_names[name] = {
            'link_year_no_x_name': str(result_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': result_links[link]['link_year_no'], 
            'year_no': result_links[link]['stream_election_info']['year_no'], 
            'stream_no': result_links[link]['stream_election_info']['stream_no'], 
            'votes_cnt': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['votes_cnt'],
            'elected_flg': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['elected_flg'],
            'link': link
        }

result_names_df =  pd.DataFrame(result_names.items(), columns=['name', 'params'])
result_names_df_2 = pd.concat([result_names_df.drop(['params'], axis=1), result_names_df['params'].apply(pd.Series)], axis=1)
result_names_df_2.head(10)


,name,link_year_no_x_name,link_year_no,year_no,stream_no,votes_cnt,elected_flg,link
0,Хапкин Артём Владимирович,2014_Хапкин_Артём_Владимирович,2014,4,1,32,1,https://vk.com/wall-33327655_2240
1,Тулякова Анна Владимировна,2014_Тулякова_Анна_Владимировна,2014,4,1,19,1,https://vk.com/wall-33327655_2240
2,Матвеев Дмитрий Александрович,2014_Матвеев_Дмитрий_Александрович,2014,4,1,45,1,https://vk.com/wall-33327655_2240
3,Бунеев Артём Валерьевич,2014_Бунеев_Артём_Валерьевич,2014,4,2,44,1,https://vk.com/wall-33327655_2241
4,Борисов Василий Александрович,2014_Борисов_Василий_Александрович,2014,4,2,49,1,https://vk.com/wall-33327655_2241
5,Быстрицкая Василина Васильевна,2014_Быстрицкая_Василина_Васильевна,2014,4,3,19,1,https://vk.com/wall-33327655_2242
6,Мамедов Рамин Кямран Оглы,2014_Мамедов_Рамин_Кямран_Оглы,2014,4,3,31,1,https://vk.com/wall-33327655_2242
7,Антонов Сергей Валентинович,2014_Антонов_Сергей_Валентинович,2014,5,2,25,1,https://vk.com/wall-33327655_2243
8,Митькина Мария Александровна,2014_Митькина_Мария_Александровна,2014,5,2,33,1,https://vk.com/wall-33327655_2243
9,Мылзенова Дарима Баировна,2014_Мылзенова_Дарима_Баировна,2014,5,2,24,1,https://vk.com/wall-33327655_2243


In [ ]:
# ТЕПЕРЬ НАДО СМЁРДЖИТЬ СЛОВАРИ ПО НОМЕРУ ПОТОКА
# или по кандидатам?

In [9]:
import pandas as pd

In [10]:
# pd.DataFrame(announcement_links.items(), columns=['link', 'link_info'])

In [11]:
pd.DataFrame(result_links.items(), columns=['link', 'link_info'])

# 

,link,link_info
0,https://vk.com/wall-33327655_2215,"{'link_type': 'election', 'link_subtype': 'res..."
1,https://vk.com/wall-33327655_2216,"{'link_type': 'election', 'link_subtype': 'res..."
2,https://vk.com/wall-33327655_2240,"{'link_type': 'election', 'link_subtype': 'res..."
3,https://vk.com/wall-33327655_2241,"{'link_type': 'election', 'link_subtype': 'res..."
4,https://vk.com/wall-33327655_2242,"{'link_type': 'election', 'link_subtype': 'res..."
5,https://vk.com/wall-33327655_2243,"{'link_type': 'election', 'link_subtype': 'res..."
6,https://vk.com/wall-33327655_2246,"{'link_type': 'election', 'link_subtype': 'res..."
7,https://vk.com/wall-33327655_2250,"{'link_type': 'election', 'link_subtype': 'res..."
8,https://vk.com/wall-33327655_2251,"{'link_type': 'election', 'link_subtype': 'res..."
9,https://vk.com/wall-33327655_2252,"{'link_type': 'election', 'link_subtype': 'res..."


In [56]:
# pd.DataFrame.from_records(result_links).reset_index().rename(columns={'index':'Sector'})

In [11]:
result_links_df = pd.DataFrame.from_dict(result_links, orient='index').reset_index().rename(columns={'index':'link'})
result_links_df[['link','link_date','link_time','stream_election_info']]

# НЕ ХВАТАЕТ КОЛОНКИ С НОМЕРОМ КУРСА-ПОТОКА, ПОЧЕМУ ЕЕ НЕТ?

,link,link_date,link_time,stream_election_info
0,https://vk.com/wall-33327655_2215,2014-11-17,21:06,"{'year_stream_no': '', 'year_no': 0, 'stream_n..."
1,https://vk.com/wall-33327655_2216,2014-11-17,21:21,"{'year_stream_no': '', 'year_no': 0, 'stream_n..."
2,https://vk.com/wall-33327655_2240,2014-11-21,21:45,"{'year_stream_no': '4.1', 'year_no': 4, 'strea..."
3,https://vk.com/wall-33327655_2241,2014-11-21,21:50,"{'year_stream_no': '4.2', 'year_no': 4, 'strea..."
4,https://vk.com/wall-33327655_2242,2014-11-21,22:00,"{'year_stream_no': '4.3', 'year_no': 4, 'strea..."
5,https://vk.com/wall-33327655_2243,2014-11-21,22:05,"{'year_stream_no': '5.2', 'year_no': 5, 'strea..."
6,https://vk.com/wall-33327655_2246,2014-11-21,22:10,"{'year_stream_no': '3.4', 'year_no': 3, 'strea..."
7,https://vk.com/wall-33327655_2250,2014-11-21,22:15,"{'year_stream_no': '1.1', 'year_no': 1, 'strea..."
8,https://vk.com/wall-33327655_2251,2014-11-21,22:20,"{'year_stream_no': '5.1', 'year_no': 5, 'strea..."
9,https://vk.com/wall-33327655_2252,2014-11-21,22:25,"{'year_stream_no': '1.2', 'year_no': 1, 'strea..."


In [12]:
sub_dict = result_links_df.iloc[0]['stream_election_info']
df_union_all = pd.DataFrame([sub_dict])

for i in range(1, len(result_links_df)):
#     print(result_links_df.iloc[i]['stream_election_info'])
    sub_dict = result_links_df.iloc[i]['stream_election_info']
#     print(pd.DataFrame.from_dict(sub_dict, orient='index').reset_index())   # .rename(columns={'index':'link'})    
#     print(pd.DataFrame.from_dict(sub_dict, orient='index'))
#     print(pd.DataFrame([sub_dict]))
    
    df_union_all = pd.concat([df_union_all, pd.DataFrame([sub_dict])])

df_election_results = df_union_all.reset_index()
df_election_results

,index,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers,dict_nominated_with_votes
0,0,,0,0,135,87,0,7,[Глунчадзе Ираклием],{'Суркова Анастасия Юрьевна': {'votes_cnt': 53...
1,0,,0,0,105,86,0,8,[Митькиной Марией],{'Дорман Елена Вячеславовна': {'votes_cnt': 71...
2,0,4.1,4,1,102,52,0,1,"[Анастасией Сурковой, Татьяной Тюриной]",{'Хапкин Артём Владимирович': {'votes_cnt': 32...
3,0,4.2,4,2,102,54,0,0,"[Еленой Дорман, Татьяной Тюриной]","{'Бунеев Артём Валерьевич': {'votes_cnt': 44, ..."
4,0,4.3,4,3,82,43,0,9,[Алексеем Матвеевым],{'Быстрицкая Василина Васильевна': {'votes_cnt...
5,0,5.2,5,2,70,35,0,0,[Алексеем Матвеевым],{'Антонов Сергей Валентинович': {'votes_cnt': ...
6,0,3.4,3,4,48,24,0,0,[Марией Митькиной],{'Чеботаева Виктория Алексеевна': {'votes_cnt'...
7,0,1.1,1,1,166,129,10,5,"[Ильёй Денисовым, Анастасией Сурковой]","{'Нечаев Никита Сергеевич': {'votes_cnt': 19, ..."
8,0,5.1,5,1,96,50,0,0,[Марией Митькиной],{'Ламтюгин Алексей Валерьевич': {'votes_cnt': ...
9,0,1.2,1,2,136,95,4,6,"[Ираклием Глунчадзе, Анастасией Сурковой]","{'Нагорных Яна Валерьевна': {'votes_cnt': 46, ..."


In [14]:
sub_sub_dict = df_election_results.iloc[0]['dict_nominated_with_votes']
# df_union_all = pd.DataFrame.from_dict(sub_sub_dict).reset_index()
df_union_all = pd.DataFrame(sub_sub_dict.items())

for i in range(1, len(df_election_results)):
    sub_sub_dict = df_election_results.iloc[i]['dict_nominated_with_votes']
    
#     df_union_all = pd.concat([df_union_all, pd.DataFrame.from_dict(sub_sub_dict).reset_index()])
    df_union_all = pd.concat([df_union_all, pd.DataFrame(sub_sub_dict.items())])

df_personal_election_results = df_union_all.reset_index().rename(columns={0:'name', 1:'params'})
df_personal_election_results = df_personal_election_results.drop('index', 1)
df_personal_election_results.head()

,name,params
0,Суркова Анастасия Юрьевна,"{'votes_cnt': 53, 'elected_flg': 1}"
1,Денисов Илья Олегович,"{'votes_cnt': 61, 'elected_flg': 1}"
2,Тюрина Татьяна Игоревна,"{'votes_cnt': 48, 'elected_flg': 1}"
3,Дорман Елена Вячеславовна,"{'votes_cnt': 71, 'elected_flg': 1}"
4,Оськин Александр Андреевич,"{'votes_cnt': 45, 'elected_flg': 1}"


In [27]:
# sub_sub_sub_dict = df_personal_election_results.iloc[0]['params']
# df_union_all = pd.DataFrame(sub_sub_sub_dict.items())

# for i in range(1, len(df_personal_election_results)):
#     sub_sub_sub_dict = df_personal_election_results.iloc[i]['params']
#     df_union_all = pd.concat([df_union_all, pd.DataFrame(sub_sub_sub_dict.items())])

# df_personal_election_results2 = df_union_all.reset_index()   #.rename(columns={0:'name', 1:'params'})
# df_personal_election_results2


# df_personal_election_results2 = pd.DataFrame.from_dict(df_personal_election_results, orient='index').reset_index()  #.rename(columns={'index':'link'})
df_union_all = pd.DataFrame([df_personal_election_results['params'][0]])
for i in range(1, len(df_personal_election_results)):
    sub_sub_sub_dict = df_personal_election_results.iloc[i]['params']
    df_union_all = pd.concat([df_union_all, pd.DataFrame([df_personal_election_results['params'][i]])])
df_personal_election_results2 = df_union_all
df_personal_election_results2

,votes_cnt,elected_flg
0,53,1
0,61,1
0,48,1
0,71,1
0,45,1
0,32,1
0,19,1
0,45,1
0,44,1
0,49,1


In [42]:
sub_dict = result_links_df.iloc[2]['stream_election_info']
sub_dict

{'year_stream_no': '4.1',
 'year_no': 4,
 'stream_no': 1,
 'stream_size': 102,
 'voted_cnt': 52,
 'invalid_cnt': '0',
 'NOTA_cnt': '1',
 'organizers': ['Анастасией Сурковой', 'Татьяной Тюриной'],
 'dict_nominated_with_votes': {'Хапкин Артём Владимирович': {'votes_cnt': 32,
   'elected_flg': 1},
  'Тулякова Анна Владимировна': {'votes_cnt': 19, 'elected_flg': 1},
  'Матвеев Дмитрий Александрович': {'votes_cnt': 45, 'elected_flg': 1}}}

In [48]:
# pd.DataFrame.from_records(sub_dict)
# pd.DataFrame.from_dict(sub_dict, index=index, columns=columns)
# pd.DataFrame(sub_dict.items())
pd.DataFrame([sub_dict])

# pd.DataFrame(result_links.items(), columns=['link', 'link_info'])


,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers,dict_nominated_with_votes
0,4.1,4,1,102,52,0,1,"[Анастасией Сурковой, Татьяной Тюриной]",{'Хапкин Артём Владимирович': {'votes_cnt': 32...


In [ ]:

'''
ДАЛЕЕ НЕАКТУАЛЬНО
'''
 



In [ ]:
# образец выворачивания словаря: 
# d = {1: {"A": "i", "B": "ii", "C": "i"}, 2: {"B": "i", "C": "ii"}, 3: {"A": "iii"}}
# keys = ['A', 'B', 'C']
# e = {key: {k: d[k][key] for k in d if key in d[k]} for key in keys}
# e
# {'A': {1: 'i', 3: 'iii'}, 'B': {1: 'ii', 2: 'i'}, 'C': {1: 'i', 2: 'ii'}}

In [531]:
# образец:

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2225')
# link_info

# for key in link_info:
#     list_of_lines = link_info[key]['list_of_lines']
# list_of_lines

In [532]:
# nominated_dict = nominated_by_list(list_of_lines)
# nominated_dict

In [455]:
# примеры:
    
# 33327655_2225 - announcemen
# 33327655_2252 - result
# 33327655_8800 - коммент

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2252')
# link_info

In [456]:
# в интервале ссылок составили словарь для всех постов про выборы

# election_links = {}

# for num in range(2200, 2250):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if d['link_type'] == 'election':
#             election_links = {**election_links, **test_dict}
        
# election_links

In [457]:
# not_found_links = {}

# for num in range(2200, 2230):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if d['link_type'] == 'page not found':
#             not_found_links = {**not_found_links, **test_dict}
        
# not_found_links

In [458]:
# other_links = {}

# for num in range(2200, 2230):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if not (d['link_type'] in ('election', 'page not found')):
#             other_links = {**other_links, **test_dict}
        
# other_links

In [450]:
# работа с постом с РЕЗУЛЬТАТАМИ выборов

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2252')
# link_info

In [451]:
# пока некорректное присваивание в цикле, но для примера с 1 ссылкой сойдет

# for d in link_info.values():
#     list_of_lines = d['list_of_lines']
# list_of_lines

In [155]:
# функция, которая по адресу ссылки возвращает список строк текста

# <div class="wi_body">
#     <div class="pi_text">

def vk_post_lines_by_link(link_vk):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    # ВЫТАЩИЛ НУЖНЫЙ БЛОК ТЕКСТА ПОСТРОЧНО
    
    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    if wi_body:
        pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
        text_lines = [line for line in pi_text.stripped_strings]
    else:
        text_lines = []
        
    return text_lines

In [288]:
link_lines_2 = vk_post_lines_by_link_2('https://vk.com/wall-33327655_2225')
# 33327655_2225 - announcemen
# 33327655_2252 - result
link_lines_2


{'https://vk.com/wall-33327655_2225': {'type': 'announcement',
  'list of lines': ['#выборы@stud_mm',
   'Насыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.',
   'Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».',
   '1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)',
   '— НАГОРНЫХ Яна Валерьевна, группа 108',
   '«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и яркая внеучебная жизнь, вот и мне хочется участвовать в организации т

In [134]:
# функция, которая по адресу ссылки возвращает дату поста (или год?)
# <div class="wi_info">
#     <span class="wi_date">

def vk_post_date_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    wi_info = soup.find('div', attrs={'class': 'wi_info'})
    if wi_info:
        wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
        date_list = [date for date in wi_date.stripped_strings]
        return date_list[0]
    else:
        return ''

def vk_post_year_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    wi_info = soup.find('div', attrs={'class': 'wi_info'})
    if wi_info:
        wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
        date_list = [date for date in wi_date.stripped_strings]
        return re.findall(r'[\d]{4}', date_list[0])[0]
    else:
        return ''


In [135]:
link_date = vk_post_date_by_link('https://vk.com/wall-33327655_2220')
print(link_date)
link_year = vk_post_year_by_link('https://vk.com/wall-33327655_2220')
print(link_year)

18 ноя 2014 в 23:10
2014


In [88]:
# функция, которая по адресу ссылки возвращает список строк текста

# <div class="wi_body">
#     <div class="pi_text">

def vk_post_lines_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    # ВЫТАЩИЛ НУЖНЫЙ БЛОК ТЕКСТА ПОСТРОЧНО
    
    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    if wi_body:
        pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
        return [line for line in pi_text.stripped_strings] 
    else:
        return []
    

#     вариант на случай, если блоков текста несколько, но у нас он один (если не смотреть в комменты)
#     text_by_lines = []
#     for wi_body in soup.find_all('div', attrs={'class': 'wi_body'}):
#         pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
#         text_by_strings = [line for line in pi_text.stripped_strings]
#         text_by_lines.append(text_by_strings)
#     if len(text_by_lines) > 0:
#         return text_by_lines[0]
#     else:
#         return []

In [459]:
# list_of_lines = vk_post_lines_by_link('https://vk.com/wall-33327655_2220')
# print(list_of_lines)

In [12]:
# сформированный руками список кандидатов из этого текста

list_of_nominated = [
    'НЕЧАЕВ Никита Сергеевич',
    'МИЗРАКИ Вячеслав Алексеевич',
    'ЯШИН Ростислав Игоревич',
    'ЗМИЕВСКИЙ Леонид Вячеславович',
    'ТАИБОВ Али Зайнудин Гусейнович',
    'КАРЯКИН Игорь Андреевич'
]

In [50]:
# словарик для индексов строк, соответствующих началу программы кандидата

dict_of_nominated = {}

for line in list_of_lines:
    for name in list_of_nominated:
        if line.find(name) > 0:
            dict_of_nominated[name] = list_of_lines.index(line)
#             print(line,name)
# print(dict_of_nominated)

# на всякий случай: отсортировать словарь по ключу
dict(sorted(dict_of_nominated.items(), key=lambda item: item[1]))


{'НЕЧАЕВ Никита Сергеевич': 4,
 'МИЗРАКИ Вячеслав Алексеевич': 11,
 'ЯШИН Ростислав Игоревич': 13,
 'ЗМИЕВСКИЙ Леонид Вячеславович': 15,
 'ТАИБОВ Али Зайнудин Гусейнович': 17,
 'КАРЯКИН Игорь Андреевич': 23}

In [51]:
# первый и последний индексы программы кандидата в списке

name_list = list(dict_of_nominated.keys())
# print(name_list)

start_index_list = list(dict_of_nominated.values())
# print(start_index_list)

last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
last_index_list.append(len(list_of_lines)-1)
# print(last_index_list)

dict_of_nominated_upd = {}

for name in dict_of_nominated:
    dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
print(dict_of_nominated_upd)

{'НЕЧАЕВ Никита Сергеевич': [4, 10], 'МИЗРАКИ Вячеслав Алексеевич': [11, 12], 'ЯШИН Ростислав Игоревич': [13, 14], 'ЗМИЕВСКИЙ Леонид Вячеславович': [15, 16], 'ТАИБОВ Али Зайнудин Гусейнович': [17, 22], 'КАРЯКИН Игорь Андреевич': [23, 34]}


In [49]:
# новый словарь: имя: номер группы, текст программы


dict_of_nominated_full = {}

for name in dict_of_nominated_upd:
    original_name = list_of_lines[dict_of_nominated_upd[name][0]]
    tail_name = re.sub(name, '', original_name)
    academic_group_no = re.sub('\D', '', tail_name)

    program_lines = []
    for k in range(dict_of_nominated_upd[name][0]+1,dict_of_nominated_upd[name][1]+1):
        program_lines.append(list_of_lines[k])
    program_txt = '\n'.join(program_lines)
    
#     print(name+':')
#     print(academic_group_no)
#     print(program_txt)
#     print('\n')
    
    dict_of_nominated_full[name] = [academic_group_no, program_txt]

dict_of_nominated_full

{'НЕЧАЕВ Никита Сергеевич': ['102',
  '«Я постоянно что-то пытаюсь организовать в кругу своих знакомых, нередко получается весьма хорошо (например, летом было 3 встречи абитуриентов МГУ), поэтому, если попаду в Студсовет, то моими главными задачами будут: во-первых, представление интересов первого курса МехМата МГУ в тех рамках, которые мне будут доступны и, во-вторых, организация небольших ивентов на МехМате (есть уже идея с вечером настольных игр в ГЗ в какую-нибудь пятницу и еще пара пока что сырых проектов)\nКак-то так:)\nО себе?\nvk.com/nnsf0sker\n17 лет, русский, учился на дому, снимался в кино, играл на барабанах и много разных странностей творил :)\nP. S ВКонтакте имя измененно чисто случайно, но Валентин - это Никита)»'],
 'МИЗРАКИ Вячеслав Алексеевич': ['101',
  '«вступить в рабочую группу Праздники»'],
 'ЯШИН Ростислав Игоревич': ['104',
  '«Хочу быть полезным своему университету.»'],
 'ЗМИЕВСКИЙ Леонид Вячеславович': ['103',
  '«Студенческий совет занимается очень важной ра

In [60]:
# извлекаем номера курса и потока

year_stream_line = list_of_lines[min(start_index_list)-1]
year_stream_line = re.sub('\D', '', year_stream_line)
year_no = year_stream_line[0]
stream_no = year_stream_line[1]
year_stream_no = year_no + '.' + stream_no

year_stream_no

'1.1'

In [65]:
# 33327655_2225 - пост
# 33327655_2229 - коммент
# 33327655_15000 - не существует

link_testt = 'https://vk.com/wall-33327655_15000'
req_testt = requests.get(link_testt)

req_testt.status_code

200

In [ ]:
# список ссылок на посты про выборы

In [69]:
test_link = 'https://vk.com/wall-33327655_2225'

test_list = vk_post_lines_by_link(test_link)
if test_list[0] == '#выборы@stud_mm':
    print('Yes')

Yes


In [289]:
# election_links = []

# for num in range(2200, 2300):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_list = vk_post_lines_by_link(test_link)
    
#     if test_list == []:
#         pass
#     elif '#выборы@stud_mm' in test_list:
#         election_links.append(test_link)
        
# election_links

In [290]:
# то же самое, но мб побыстрее

# election_links_2 = ['https://vk.com/wall-33327655_' + str(num) for num in range(2200, 2300) if '#выборы@stud_mm' in vk_post_lines_by_link('https://vk.com/wall-33327655_' + str(num))]
# election_links_2


In [291]:
# год по списку ссылок

# election_years_2 = [vk_post_year_by_link(link) for link in election_links_2]
# election_years_2

In [235]:
req_announcement = requests.get('https://vk.com/wall-33327655_2225')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="#выборы@stud_mm" />
        <meta property="og:description" content="#выборы@stud_mm" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-

In [236]:
# заведомо бракованная ссылка выглядит так:

req_announcement = requests.get('https://vk.com/wall-33327655_2225')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="#выборы@stud_mm" />
        <meta property="og:description" content="#выборы@stud_mm" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-

In [207]:
# ссылка на коммент выглядит так (как ссылка на родительский пост):

# когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
# когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />

req_announcement = requests.get('https://vk.com/wall-33327655_8800')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="Приостановка сбора заявлений на ПГАС" />
        <meta property="og:description" content="Приостановка сбора заявлений на ПГАС" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa

In [95]:
# вытаскивает весь текст, но так пока не нужно.

from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://vk.com/wall-33327655_2225"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out
# get text
text = soup.get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)
print(text)

#выборы@stud_mm Насыщенная выборная.. | Студенческий совет мехмата МГУ | ВКонтакте
Запись на стене
Студенческий совет мехмата МГУ
19 ноя 2014 в 21:07
#выборы@stud_mmНасыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)— НАГОРНЫХ Яна Валерьевна, группа 108«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и яркая внеучебная жизнь, вот и мне хоче

In [98]:
# КОГДА ДОПИСАЛИ КАНДИДАТОВ В КОММЕНТ (единичный случай, забить)


from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://vk.com/wall-33327655_2225"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

text_by_strings = [text for text in soup.stripped_strings]
print(text_by_strings)

# for line in text_by_strings:
    

['#выборы@stud_mm Насыщенная выборная.. | Студенческий совет мехмата МГУ | ВКонтакте', 'Запись на стене', 'Студенческий совет мехмата МГУ', '19 ноя 2014 в 21:07', '#выборы@stud_mm', 'Насыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.', 'Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».', '1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)', '— НАГОРНЫХ Яна Валерьевна, группа 108', '«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и ярка